In [1]:
############モジュールインポート###############
!pip install cython
!pip install onnxruntime #-gpu
!pip install insightface
from insightface.app import FaceAnalysis
from google.colab import drive
import os
import glob
import json
import numpy as np
from PIL import ImageFile
from PIL import Image #画像処理ライブラリ
import cv2 #動画編集ライブラリ
# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True
from google.colab.patches import cv2_imshow # パッチファイルインポート

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 4.5 MB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 439 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.5 MB 53.9 MB/s 
     |████████████████████████████████| 1.0 MB 56.1 MB/s 
  Created wheel for insightface: filename=insightface-0.7.2-cp38-cp38-linux_x86_64.whl size=992165 sha256=5ebe94ab28d6add6572bc60a4b38d0d0e53b3e6ae6177b7795b637d6dbf65d14
  Stored in director

In [2]:
##############マウント#####################
drive.mount('/content/drive/') #マウント
#os.chdir('/drive') #MyDriveにディレクトリ移動
%cd /content/drive/MyDrive
root_path = os.getcwd()
print("root_path : " + str(root_path)) #現在のディレクトリ確認

for name in glob.glob("*"): #ディレクトリ直下のフォルダとファイルを全参照
  print("directries : " + str(name))

print(type(glob.glob("*")))

Mounted at /content/drive/
/content/drive/MyDrive
root_path : /content/drive/MyDrive
directries : j_py02.json
directries : Colab
directries : colabo_pro
directries : datasets
directries : sample_100.jpg
directries : test2.jpg
<class 'list'>


In [4]:
#############変数定義######################
#ファイルの取得、ファイル名の定義
classes = ["FAKE", "REAL"]
num_classes = len(classes)
image_size = 64
num_testdata = 25
flame_num = 10 #処理するフレイム数の定義
path_r = r'/content/drive/MyDrive/datasets/Deepfake_detection_Challenge'
files_s = glob.glob(path_r + r'/train_sample_videos/*.mp4')
flip_num = 80 #テストデータの枚数
#files_t = glob.glob(path_r + r'/test_videos/*.mp4')



#jsonファイルの取得
j_ws = open(path_r + "/train_sample_videos/metadata.json","r") #jsonの読み込みと辞書型への変形
#print(j_w2)
label_dic = json.load(j_ws)
print(files_s)
print(label_dic)
print(type(label_dic))
print(len(label_dic))

['/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/abarnvbtwb.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/abqwwspghj.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/acxnxvbsxk.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/acifjvzvpm.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/aczrgyricp.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/abofeumbvv.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/aapnvogymq.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/acqfdwsrhi.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/acxwigylke.mp4', '/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/aagfhgtpmv.mp4', '/content

In [5]:
##########動画・フレームの取得、顔の切り出し関数#########
def save_frame(video_path, frame_num): #引数は順番に、動画ファイルのパス、注目フレイムの場所
  cap = cv2.VideoCapture(video_path)
  if not cap.isOpened():
    return
  
  #os.makedirs(os.path.dirname(result_path), exist_ok=True) #出力用のディレクトリを定義
  cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
  ret, frame = cap.read() #多分成功するとretにbool形式でTrueが返される。frameがndarrayの中身

  img = frame
  app = FaceAnalysis()
  app.prepare(ctx_id=0, det_size=(640, 640))

  faces = app.get(np.asarray(img)) #InsightFaceを使用して顔の情報を取得
  #print("faces:" + str(len(faces)))#print(type(faces))#print(faces[0]['landmark_2d_106']
  
  if len(faces) == 1:
    #輪郭座標一覧から最大値と最小値を取得したうえで切り出しを行う
    i0_l = faces[0]['landmark_2d_106'][0][0]
    i1_l = faces[0]['landmark_2d_106'][0][1]
    i0_h = faces[0]['landmark_2d_106'][0][0]
    i1_h = faces[0]['landmark_2d_106'][0][1]
    
    print(type(i0_l))
    
    for i in faces[0]['landmark_2d_106']:
      i0 = i[0]
      i1 = i[1]
      if i0_l < i0:
        i0_l = i0
      else:
        pass
      if i0_h > i0:
        i0_h = i0
      else:
        pass
      if i1_l < i1:
        i1_l = i1
      else:
        pass
      if i1_h > i1:
        i1_h = i1
      else:
        pass
      
    print([[i0_l, i0_h], [i1_l, i1_h]])
    #for number in range(0, len(face)):
    
    face_cut = img[int(i1_h):int(i1_l), int(i0_h):int(i0_l)] #この時点で失敗したら、動画⇒画像の形式変換がうまうできていない #顔部分を切り取
    face_cut = cv2.resize(face_cut, dsize=(128,128))
    #cv2.imwrite("/content/drive/MyDrive/test2.jpg", face_cut)
    ##cv2_imshow("Image", face_cut)
    return face_cut
  else:
    pass


In [6]:
#################前々処理実行################
X_train = [] #学習、テスト用の変数を定義
X_test  = []
y_train = []
y_test  = []

for file in files_s:
  file_name = os.path.basename(file)
  label_now = label_dic[str(file_name)]['label'] #※※※※習性箇所。ここのfileはパスそのものを指すので、ファイル名の取得を行う事。
  label_num = [0]
  if label_now == 'FAKE':
    pass
  else:
    label_num[0] = 1 
  for flame in range(1,flame_num + 1):
    print([file, flame])
    face_cut = save_frame(file, flame) #print('Length : ', len(face_cut))

    if type(face_cut) != np.ndarray:
      print('No faces.')
    else:
      face_list = face_cut.tolist()
      
      #画像などを代入(関数を定義しておく)
      rx = np.random.rand()
      if rx <= 0.2:
        X_test.append(face_list)
        y_test.append(label_num)
      else:
        X_train.append(face_list)
        y_train.append(label_num)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(f"x_train shape: {X_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {X_test.shape} - y_test shape: {y_test.shape}")
#ここから書き足し
print(type(X_train))
print(type(X_test))

['/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/abarnvbtwb.mp4', 1]
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 73119.40KB/s]
/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


/usr/local/lib/python3.8/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


ストリーミング出力は最後の 5000 行に切り捨てられました。
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
<class 'numpy.float32'>
[[1005.9079, 922.7902], [267.0692, 179.57869]]
['/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/ehevsxtecd.mp4', 3]
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.

In [ ]:
print("=================================================================")
print(y_train)
print("=================================================================")
print(y_test)
print("=================================================================")
print(X_train)
print("=================================================================")
print(X_test)

[[1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(X_train[0])

[]


In [1]:
#ndarrayをファイル化して保存(いちいち読み込むと非常にメンドクサイため)
path_array = '/content/drive/MyDrive/datasets/ndarray'
np.save(path_array + r'/X_test_5_normal', X_test) #ファイル名は、変数名_フレーム数_モードとする。
np.save(path_array + r'/X_train_5_normal', X_train) #モードは今のところ、normal,arcface,lbp_色空間を予定している。
np.save(path_array + r'/y_test_5_normal', y_test)
np.save(path_array + r'/y_train_5_normal', y_train)

NameError: ignored

# **ここから実験用コード**

In [ ]:
#実験用、関数
def test_function(a,b,c):
  d = a + b
  e = b * c
  return [d,e]

a = 3
b = 4
c = 5

d = test_function(a,b,c)[0]
e = test_function(a,b,c)[1]
print(d)
print(e)

class TestClass:
  
  def __init__(self, x1, x2, x3): #初期値を入力にする。
    self.x1 = x1
    self.x2 = x2
    self.x3 = x3
  
  def function1(self): #自身が持っている数値__inita__で定義した数値を使って計算できる。
    x4 = x1 + x3
    return x4
  
  def function2(self,x4):
    self.x4 = x4
    x5 = x1 + x3 - x4
    return x5

x1 = 10
x2 = 11
x3 = 13

x4 = TestClass(x1, x2, x3) #TestClassをインスタンス化し、かっこ内の数値を代入
print(x4.function1()) #TestClass内の関数function1を呼び出して先ほど入力した数値で実行
k = x4.function1() #TestClassのfunction1の返り値をkに代入
print(x4.function2(k))

#モジュールなうのメゾットを検索
from inspect import getmembers, isfunction
print(getmembers(FaceAnalysis, isfunction))

7
20
23
0


In [ ]:
#顔検出と切り出し
cap = cv2.VideoCapture('/content/drive/MyDrive/datasets/Deepfake_detection_Challenge/train_sample_videos/afoovlsmtx.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
ret, frame = cap.read()

img = frame

app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))

faces = app.get(np.asarray(img))
print("faces:" + str(len(faces)))
print(type(faces))
print(faces[0]['landmark_2d_106'])

i0_l = faces[0]['landmark_2d_106'][0][0]
i1_l = faces[0]['landmark_2d_106'][0][1]
i0_h = faces[0]['landmark_2d_106'][0][0]
i1_h = faces[0]['landmark_2d_106'][0][1]
print([[i0_l, i0_h], [i1_l, i1_h]])

print(type(i0_l))

for i in faces[0]['landmark_2d_106']:
  i0 = i[0]
  i1 = i[1]
  if i0_l < i0:
    i0_l = i0
  else:
    pass
  if i0_h > i0:
    i0_h = i0
  else:
    pass
  if i1_l < i1:
    i1_l = i1
  else:
    pass
  if i1_h > i1:
    i1_h = i1
  else:
    pass

print(i0_l)
print(i1_l)
print(i0_h)
print(i1_h)
print([[i0_l, i0_h], [i1_l, i1_h]])
#顔部分を切り取る
#for number in range(0, len(face)):

face_cut = frame[int(i1_h):int(i1_l), int(i0_h):int(i0_l)] #この時点で失敗したら、動画⇒画像の形式変換がうまうできていない
print(type(face_cut))
cv2.imwrite("/content/drive/MyDrive/test2.jpg", face_cut)

##cv2_imshow("Image", face_cut)

/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


/usr/local/lib/python3.8/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


faces:1
<class 'list'>
[[438.67944 988.4893 ]
 [333.78427 791.66547]
 [347.9993  931.76544]
 [354.9885  945.8764 ]
 [364.25458 958.5748 ]
 [375.55103 969.2574 ]
 [388.89615 977.6777 ]
 [403.9726  983.7023 ]
 [420.62686 987.3231 ]
 [331.91055 807.27325]
 [330.84625 822.8222 ]
 [330.5983  838.3754 ]
 [331.19196 853.94653]
 [332.8204  869.6223 ]
 [335.51212 885.35   ]
 [338.68417 901.0709 ]
 [342.7083  916.6766 ]
 [553.3591  784.61395]
 [542.4734  935.2605 ]
 [534.07245 949.6723 ]
 [522.8998  962.179  ]
 [509.24982 972.23486]
 [493.6383  979.78217]
 [476.3241  984.9248 ]
 [457.86795 987.81226]
 [555.8808  801.15894]
 [557.5913  817.773  ]
 [558.3758  834.4729 ]
 [558.2926  851.2654 ]
 [557.31757 868.29694]
 [555.446   885.5382 ]
 [552.72406 902.6791 ]
 [548.5535  919.42053]
 [383.8122  803.54645]
 [384.42935 796.8896 ]
 [362.85715 801.2425 ]
 [372.77692 803.4651 ]
 [394.89468 800.9965 ]
 [384.43237 796.8787 ]
 [404.94235 797.4326 ]
 [383.15366 791.17847]
 [371.7579  794.6539 ]
 [394.95288

True

In [ ]:
if type(face_cut) == np.ndarray:
  print('Yes')
else:
  print('No')

Yes


In [ ]:
hoge10 = 60
hoge11 = 10
if hoge10 < 80 and hoge11 <= 80:
  print(True)
else:
  print(False)

True


In [ ]:
r = np.random.rand()
print(r)
print(5 / 10)
bit = np.random.randint(0, 2, 10)
print(bit)
hoge_list =[0,0,0,0,0]
if len(hoge_list) == 0:
  print('No parametor')
else:
  print(len(hoge_list))

hoge12 = [1,2,3,4,5]
print(type(hoge12))
hoge12 = np.array(hoge12)
print(type(hoge12))

0.36000533149116665
0.5
[1 0 1 1 1 0 0 0 0 0]
5
<class 'list'>
<class 'numpy.ndarray'>


In [ ]:
def HOGE(hoge16):
  print(hoge16)
  print('Done')
  return [5,6,7,8]

hoge15 = HOGE(1)
if len(hoge15) == 0:
  print('No subject')
else:
  print('Fooo')
  print(hoge15)

1
Done
Fooo
[5, 6, 7, 8]


# **ここから引用コード**

In [ ]:
#numpy型に成形、ラベルをy_trainに代入
from PIL import Image #画像処理ライブラリ
import cv2 #動画編集ライブラリ
import os, glob
import numpy as np
from PIL import ImageFile
# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True

# indexを教師ラベルとして割り当てるため、0にはdogを指定し、1には猫を指定
classes = ["FAKE", "REAL"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            # angleに代入される値
            # -20
            # -15
            # -10
            #  -5
            # 0
            # 5
            # 10
            # 15
            # 画像を5度ずつ回転
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index) # indexを教師ラベルとして割り当てるため、0にはdogを指定し、1には猫を指定

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

# **保存したいコード**

In [ ]:
###############テスト振り分けbit作成##################
class Input_data():

  def __init__(self, bits, flip_num, image_num): #左から順番に、生成するビット列、振り分ける数、全体の画像数
    self.bits = bits
    self.flip_num = flip_num
    self.image_num = image_num
    self.count = 0
  
  def make_bit(self):
    while True:
      if len(bits) == image_num:
        if count < flip_num:
          bit = np.random.randint(0, image_num - 1)
          bits[bit] = 1
        else:
          print('count: ', count)
          print('bits: ', bits)
          print('length: ', len(bits))
          break
      else:
        rx = np.random.rand()
        if count < flip_num and rx <= flip_num / image_num:
          bits.append(0)
          count+=1
        else:
          bits.append(1)
      return bits
